In [105]:
# 埼玉大学データサイエンス技術研究会 
# 第6回研究会 (2024/6/14) サンプルコード
# 作成者: 平松 薫
#
# 05_acc_2.py: 最新の加速度データを取得するプログラム #
# 
# 使い方: python 05_acc_2.py
#

import os
import sys
import time
from datetime import datetime, timezone
from struct import pack, unpack
import serial
import schedule

In [153]:
def calc_crc(buf, length):
    """
    CRC-16 を計算する関数

    """
    crc = 0xFFFF
    for i in range(length):
        crc = crc ^ buf[i]
        for i in range(8):
            carrayFlag = crc & 1
            crc = crc >> 1
            if (carrayFlag == 1):
                crc = crc ^ 0xA001
    crcH = crc >> 8
    crcL = crc & 0x00FF
    return (bytearray([crcL, crcH]))

def s16(value):
    return -(value & 0x8000) | (value & 0x7fff)

def serial_write(_ser, _payload):
    """
    シリアルポートにコマンドを送信する関数
    _payload の前にヘッダと_payloadの長さを付加、後に CRC-16 を付加して送信
    """
    _command = b'\x52\x42' + pack('<H', len(_payload) + 2) + _payload
    _command = _command + calc_crc(_command, len(_command))
    _ser.write(_command)
    _ser.flush()
    time.sleep(0.1)
    return

def serial_read(_ser, _payload, timeout=0.0):
    """
    環境センサにコマンドを送信し、レスポンスを取得する関数
    _payload の前にヘッダと_payloadの長さを付加、後に CRC-16 を付加して送信
    レスポンスはシリアルポートから読み込み、そのまま返す
    """
    if len(_payload) > 0:
        serial_write(_ser, _payload)
    else:
        return b''
    
    ret = b''
    _ser_len = 0
    while _ser_len == 0:
        time.sleep(0.1)
        _ser_len = _ser.inWaiting()
    while _ser_len > 0:
        ret = ret + _ser.read(_ser_len)
        _ser_len = _ser.inWaiting()
        _tmp_timtout = 0.0
        while _ser_len == 0:
            _ser_len = _ser.inWaiting()
            _tmp_timtout = _tmp_timtout + 0.1
            # print('## loop', _tmp_timtout, _ser_len)
            time.sleep(0.1)
            if _tmp_timtout > timeout:
                break

    # ret = _ser.read(_ser_len)
    if ret[0:2] != b'\x52\x42':
        raise print("Invalid Header", ret)
    if ret[4] != 1 and ret[4] != 2:
        for i in range(len(ret)):
            print(f'({i}) {ret[i]:02x}', end=' ')
        print()
        raise print("Error Response", hex(ret[4]))
    return ret

def dump_data(_ret):
    for i in range(len(_ret)):
        print(f'({i}) {_ret[i]:02x}', end=' ')
        if i % 16 == 15:
            print()
    print()
    return

In [4]:
def led_off(_ser):
    """
    LEDを消灯する関数
    """
    print('LED OFF')
    _payload = bytearray([0x02, # Read 0x01, Write 0x02
                          0x11, 0x51, # LED設定 (0x5111 をリトルエンディアンで送信)
                          0x00, 0x00, # LED常時消灯 (0x0000 をリトルエンディアンで送信)
                          0x00, 0x00, 0x00]) # 色設定　RGB ここでは赤に設定
    ret = serial_read(_ser, _payload)
    dump_data(ret)
    return

In [5]:
def led_on(_ser):
    """
    LEDを点灯する関数
    """
    print('LED ON')
    _payload = bytearray([0x02, # Read 0x01, Write 0x02
                          0x11, 0x51, # LED設定 (0x5111 をリトルエンディアンで送信)
                          0x01, 0x00, # LEDを点灯 (0x0001 をリトルエンディアンで送信)
                          0xFF, 0xFF, 0xFF]) # 色設定　RGB ここでは白に設定
    ret = serial_read(_ser, _payload)
    dump_data(ret)
    return

In [6]:
def check_logger_status(_ser):
    """
    ロガーの状態を確認する関数
    """
    print('### check_logger_status ###')
    _payload = bytearray([0x01, # Read 0x01, Write 0x02
                         0x19, 0x51]) # Acceleration logger status (Address: 0x5119 をリトルエンディアンで送信)
    ret = serial_read(_ser, _payload)
    dump_data(ret)
    print('Logger status', ret[7], '(0x00: Waiting 0x01: Running')
    print('Running page', f'0x{ret[8]|ret[9]<<8:04x} (Range: 0x0001 to 0x2800 (1 to 10240))')
    return ret

In [159]:
def dump_acc_data(_acc_data, _page, _time):
    # print(len(_acc_data))
    for i in range(0, len(_acc_data)-1, 6):
        x = s16(_acc_data[i+1] << 8 | _acc_data[i]) * 0.1
        y = s16(_acc_data[i+3] << 8 | _acc_data[i+2]) * 0.1
        z = s16(_acc_data[i+5] << 8 | _acc_data[i+4]) * 0.1
        _timestamp = _time+(_page*32+i/6)*0.01
        print(f'{_page} {i/6} x={x:.2f} y={y:.2f} z={z:.2f}', _timestamp, datetime.fromtimestamp(_timestamp), _timestamp)
    return

In [31]:
def get_current_data(_ser):
    # Read the latest sensor data.
    payload = bytearray([0x01, # Read 0x01, Write 0x02
                        0x21, 0x50, # Latest data Long (Address: 0x5021 をリトルエンディアンで送信)
                        ])
    ret = serial_read(_ser, payload)
    keys = ["temperature", "relative_humidity", "ambient_light",
            "barometric_pressure", "sound_noise", "eTVOC", "eCO2",
            "discomfort_index", "heat_stroke", "vibration_information",
            "si_value", "pga", "seismic_intensity"]
    values = unpack('<hHHLHHHHhBHHH', ret[8:35])
    units = [0.01, 0.01, 1, 0.001, 0.01, 1, 1, 0.01, 0.01, 1, 0.1, 0.1, 0.001]
    retval = dict([ [k, v * u] for k, v, u in zip(keys, values, units)])
    retval["time_measured"] = datetime.now()
    return retval

In [39]:
def setup_sensor(_ser):
    """加速度データの記録を開始するため、timecounter をリセットする関数"""
    payload = bytearray([0x01, # Read 0x01, Write 0x02
                        0x02, 0x52, # Time setting (Address: 0x5202 をリトルエンディアンで送信)
                        ])
    ret = serial_read(_ser, payload)
    current_timecounter  = int.from_bytes(ret[7:15], 'little')
    print('current_timecounter (r)', current_timecounter)
    if current_timecounter == 0:
        payload = bytearray([0x02, # Read 0x01, Write 0x02
                            0x02, 0x52 # Time setting (Address: 0x5202 をリトルエンディアンで送信)
                            ]) + pack('<Q', 1) # UInt64 で 1 送信
        ret = serial_read(_ser, payload)
        current_timecounter  = int.from_bytes(ret[7:15], 'little')
        print('current_timecounter (w)', current_timecounter)

    payload = bytearray([0x01, # Read 0x01, Write 0x02
                        0x03, 0x52, # Memory storage interval (Address: 0x5203 をリトルエンディアンで送信)
                        ])
    ret = serial_read(_ser, payload)
    storage_interval  = int.from_bytes(ret[7:9], 'little')
    print('storage_interval (r)', storage_interval)
    if storage_interval < 3600:
        payload = bytearray([0x02, # Read 0x01, Write 0x02
                            0x03, 0x52 # Memory storage interval (Address: 0x5203 をリトルエンディアンで送信)
                            ]) + pack('<H', 3600) # UInt16 で 3600 送信
        ret = serial_read(_ser, payload)
        storage_interval  = int.from_bytes(ret[7:9], 'little')
        print('storage_interval (w)', storage_interval)


In [85]:
# シリアルポートをオープン (インストール状況・実行環境に応じて COM3 を変更)
ser = serial.Serial("COM4", 115200, serial.EIGHTBITS, serial.PARITY_NONE)
ser.reset_input_buffer()
ser.reset_output_buffer()

In [86]:
setup_sensor(ser)

current_timecounter (r) 0
current_timecounter (w) 1
storage_interval (r) 3600


In [87]:
print('### Check mode to Acceleration data logging... (Logger Status=(7)  0x00: Waiting 0x01: Running)')
ret = check_logger_status(ser)
dump_data(ret)

### Check mode to Acceleration data logging... (Logger Status=(7)  0x00: Waiting 0x01: Running)
### check_logger_status ###
(0) 52 (1) 42 (2) 08 (3) 00 (4) 01 (5) 19 (6) 51 (7) 00 (8) 00 (9) 00 (10) f7 (11) 66 
Logger status 0 (0x00: Waiting 0x01: Running
Running page 0x0000 (Range: 0x0001 to 0x2800 (1 to 10240))
(0) 52 (1) 42 (2) 08 (3) 00 (4) 01 (5) 19 (6) 51 (7) 00 (8) 00 (9) 00 (10) f7 (11) 66 


In [88]:
get_current_data(ser)

{'temperature': 25.0,
 'relative_humidity': 53.370000000000005,
 'ambient_light': 227,
 'barometric_pressure': 1019.2040000000001,
 'sound_noise': 55.04,
 'eTVOC': 141,
 'eCO2': 1330,
 'discomfort_index': 72.12,
 'heat_stroke': 21.71,
 'vibration_information': 0,
 'si_value': 0.0,
 'pga': 0.0,
 'seismic_intensity': 0.0,
 'time_measured': datetime.datetime(2024, 4, 14, 16, 15, 40, 521251)}

In [157]:
vibration_flag = False
vibration_start_time = 0
vibration_end_time = 0

ret = get_current_data(ser)
print(f'{ret["time_measured"]} ({ret["time_measured"].strftime("%Y-%m-%d %H:%M:%S.%f")}): ' + \
      f'温度 {ret["temperature"]:.2f}℃ 湿度 {ret["relative_humidity"]:.2f}% 照度 {ret["ambient_light"]:.2f}lx 騒音 {ret["sound_noise"]:.2f}dB 振動検出 {ret["vibration_information"]} 震度 {ret["seismic_intensity"]:.2f}')
  
for i in range(20):
    ret = get_current_data(ser)
    # print(f'{ret["time_measured"]} : 温度 {ret["temperature"]:.2f}℃ 湿度 {ret["relative_humidity"]:.2f}% 照度 {ret["ambient_light"]:.2f}lx 気圧 {ret["barometric_pressure"]:.2f}hPa 騒音 {ret["sound_noise"]:.2f}dB eTVOC {ret["eTVOC"]:.2f}ppb eCO2 {ret["eCO2"]:.2f}ppm 不快指数 {ret["discomfort_index"]:.2f}HI 熱中症指数 {ret["heat_stroke"]:.2f}WB SI値 {ret["si_value"]:.2f}pga {ret["pga"]:.2f}震度 {ret["seismic_intensity"]:.2f}')
    if vibration_flag == False and ret["vibration_information"] != 0:
        print(f'{ret["time_measured"]} : 温度 {ret["temperature"]:.2f}℃ 湿度 {ret["relative_humidity"]:.2f}% 照度 {ret["ambient_light"]:.2f}lx 騒音 {ret["sound_noise"]:.2f}dB 振動検出 {ret["vibration_information"]} 震度 {ret["seismic_intensity"]:.2f}')
        print("### Vibration detected", f'({ret["time_measured"]})')
        vibration_flag = True
        vibration_start_time = ret["time_measured"]

    if vibration_flag == True and ret["vibration_information"] == 0:
        print(f'{ret["time_measured"]} : 温度 {ret["temperature"]:.2f}℃ 湿度 {ret["relative_humidity"]:.2f}% 照度 {ret["ambient_light"]:.2f}lx 騒音 {ret["sound_noise"]:.2f}dB 振動検出 {ret["vibration_information"]} 震度 {ret["seismic_intensity"]:.2f}')

        # Read the current timecounter
        payload = bytearray([0x01, # Read 0x01, Write 0x02
                            0x01, 0x52, # Latest time counter (Address: 0x5201 をリトルエンディアンで送信)
                            ])
        ret = serial_read(ser, payload)
        # dump_data(ret)
        current_timecounter = int.from_bytes(ret[7:15], 'little')
        print(f'Current timecounter: {current_timecounter}')

        # Read the accelleration memory header.
        payload = bytearray([0x01, # Read 0x01, Write 0x02
                            0x3E, 0x50, # Acceleration memory data [Header] (Address: 0x503E をリトルエンディアンで送信)
                            0x00, # Acceleration data type 0x00: Earthquake data (Normal mode) 0x01: Vibration data (Normal mode)
                            0x01, # Request acceleration memory index (Range: 0x01 to 0x0A (1 to 10) *0x01: Latest data <---> 0x0A: Last data)
                            ])
        ret = serial_read(ser, payload)
        # dump_data(ret)
        # The timecounter of earthquake end.
        data_timecounter = int.from_bytes(ret[13:21], 'little')
        print(f'Earthquake end timecounter: {data_timecounter}')

        if data_timecounter != 0:
            # Calculate time of vibration end.
            vibration_end_time = datetime.now(timezone.utc).timestamp() - (current_timecounter - data_timecounter)
            print(f'vibration_start_time: {vibration_start_time} ({vibration_start_time.strftime("%Y-%m-%d %H:%M:%S.%f")})')
            print(f'vibration_end_time: {vibration_end_time} ({datetime.fromtimestamp(vibration_end_time)})')
            # print(f'vibration_end_time: {vibration_end_time} ({vibration_end_time.strftime("%Y-%m-%d %H:%M:%S.%f")})')

            _start_page = 0x0001
            _end_page = ret[7] | ret[8]<<8            
            _payload = bytearray([0x01, # Read 0x01, Write 0x02
                                0x3F, 0x50, # Acceleration memory data [Data] (Address: 0x503F)
                                0x00, # Acceleration data type (0x00: Earthquake data 0x01: Vibration data)
                                0x01, # Request acceleration memory index UInt8 0x01: Fixed value
                                0x01, 0x00, # Request page (Start page)
                                ret[7], ret[8]]) # Request page (End page)
                                # ])+ pack('<H', _end_page - 1) # Request page (End page)
            ret = serial_read(ser, _payload, timeout=1.0)
            for i in range(_end_page):
                # print(f'### page 0x{i+1:04x}')
                # dump_data(ret[i*237:(i+1)*237])
                dump_acc_data(ret[i*237+43:(i+1)*237-2], i, vibration_end_time)

            # ret = serial_read_acc(ser, _payload, _end_page, vibration_end_time)
            # print(ret)


        # Read the accelleration memory header.
        payload = bytearray([0x01, # Read 0x01, Write 0x02
                            0x3E, 0x50, # Acceleration memory data [Header] (Address: 0x503E をリトルエンディアンで送信)
                            0x01, # Acceleration data type 0x00: Earthquake data (Normal mode) 0x01: Vibration data (Normal mode)
                            0x01, # Request acceleration memory index (Range: 0x01 to 0x0A (1 to 10) *0x01: Latest data <---> 0x0A: Last data)
                            ])
        ret = serial_read(ser, payload)
        # dump_data(ret)
        # The timecounter of earthquake end.
        data_timecounter = int.from_bytes(ret[13:21], 'little')
        print(f'Vibration end timecounter: {data_timecounter}')

        if data_timecounter != 0:
            # Calculate time of vibration end.
            vibration_end_time = datetime.now(timezone.utc).timestamp() - (current_timecounter - data_timecounter)
            print(f'vibration_start_time: {vibration_start_time} ({vibration_start_time.strftime("%Y-%m-%d %H:%M:%S.%f")}) type({type(vibration_start_time)})')
            print(f'vibration_end_time: {vibration_end_time} ({datetime.fromtimestamp(vibration_end_time)})')
            print()
            # print(f'vibration_end_time: {vibration_end_time} ({vibration_end_time.strftime("%Y-%m-%d %H:%M:%S.%f")})')

            _start_page = 0x0001
            _end_page = ret[7] | ret[8]<<8            
            _payload = bytearray([0x01, # Read 0x01, Write 0x02
                                0x3F, 0x50, # Acceleration memory data [Data] (Address: 0x503F)
                                0x01, # Acceleration data type (0x00: Earthquake data 0x01: Vibration data)
                                0x01, # Request acceleration memory index UInt8 0x01: Fixed value
                                0x01, 0x00, # Request page (Start page)
                                ret[7], ret[8]]) # Request page (End page)
                                # ])+ pack('<H', _end_page - 1) # Request page (End page)
            ret = serial_read(ser, _payload, timeout=1.0)
            for i in range(_end_page):
                # print(f'### page 0x{i+1:04x}')
                # dump_data(ret[i*237:(i+1)*237])
                dump_acc_data(ret[i*237+43:(i+1)*237-2], i, vibration_end_time)

            # ret = serial_read_acc(ser, _payload, _end_page, vibration_end_time)
            # print(ret)


        vibration_flag = False

    time.sleep(1)

2024-04-14 17:50:27.050462 (2024-04-14 17:50:27.050462): 温度 22.91℃ 湿度 52.72% 照度 212.00lx 騒音 45.23dB 振動検出 0 震度 0.00
2024-04-14 17:50:29.956592 : 温度 22.89℃ 湿度 52.79% 照度 206.00lx 騒音 84.62dB 振動検出 1 震度 4.56
### Vibration detected (2024-04-14 17:50:29.956592)
2024-04-14 17:50:31.258211 : 温度 22.88℃ 湿度 53.04% 照度 217.00lx 騒音 49.67dB 振動検出 0 震度 0.00
Current timecounter: 5697
Earthquake end timecounter: 4905
vibration_start_time: 2024-04-14 17:50:29.956592 (2024-04-14 17:50:29.956592)
vibration_end_time: 1713083839.8617 (2024-04-14 17:37:19.861700)
### page 0x0001
192
0.0 x=-86.70 y=96.90 z=-953.40 1713083839.8617 2024-04-14 17:37:19.861700 1713083839.8617
1.0 x=-52.70 y=23.10 z=-983.40 1713083839.8717 2024-04-14 17:37:19.871700 1713083839.8717
2.0 x=-30.20 y=-34.90 z=-964.60 1713083839.8817 2024-04-14 17:37:19.881700 1713083839.8817
3.0 x=-18.90 y=-89.20 z=-1013.40 1713083839.8917 2024-04-14 17:37:19.891700 1713083839.8917
4.0 x=-49.00 y=-221.00 z=-960.80 1713083839.9017 2024-04-14 17:37:19.90170

In [133]:
ret = check_logger_status(ser)
_start_page = 0x0001
_end_page = ret[8] | ret[9]<<8
_payload = bytearray([0x01, # Read 0x01, Write 0x02
                      0x3F, 0x50, # Acceleration memory data [Data] (Address: 0x503F)
                      0x02, # Acceleration data type UInt8 0x02: Logger data
                      0x01, # Request acceleration memory index UInt8 0x01: Fixed value
                      0x01, 0x00, # Request page (Start page)
                      ret[8], ret[9]]) # Request page (End page)
ret = serial_read(ser, _payload)
for i in range(_end_page):
    print(f'### page 0x{i+1:04x}')
    dump_data(ret[i*237:(i+1)*237])
    dump_acc_data(ret[i*237+43:(i+1)*237-2])

### check_logger_status ###
(0) 52 (1) 42 (2) 08 (3) 00 (4) 01 (5) 19 (6) 51 (7) 00 (8) c3 (9) 01 (10) 66 (11) 56 
Logger status 0 (0x00: Waiting 0x01: Running
Running page 0x01c3 (Range: 0x0001 to 0x2800 (1 to 10240))
### page 0001
(0) 52 (1) 42 (2) e9 (3) 00 (4) 01 (5) 3f (6) 50 (7) 01 (8) 00 (9) 00 (10) 00 (11) 00 (12) 00 (13) 00 (14) 00 (15) 00 
(16) 00 (17) 00 (18) 00 (19) 00 (20) 00 (21) cb (22) 08 (23) 5c (24) 18 (25) ce (26) 00 (27) 19 (28) 7f (29) 0f (30) 00 (31) 4d 
(32) 15 (33) 27 (34) 00 (35) 91 (36) 02 (37) 27 (38) 1b (39) 19 (40) 08 (41) ff (42) ff (43) 35 (44) 07 (45) f6 (46) 06 (47) 58 
(48) db (49) 10 (50) 07 (51) 44 (52) 07 (53) 58 (54) db (55) 10 (56) 07 (57) 44 (58) 07 (59) 58 (60) db (61) ea (62) 06 (63) 44 
(64) 07 (65) 58 (66) db (67) 10 (68) 07 (69) 1d (70) 07 (71) 33 (72) db (73) 35 (74) 07 (75) 1d (76) 07 (77) e8 (78) da (79) 5c 
(80) 07 (81) 44 (82) 07 (83) 0e (84) db (85) 5c (86) 07 (87) 44 (88) 07 (89) e8 (90) da (91) 35 (92) 07 (93) 44 (94) 07 (95) 33 
(96

In [160]:
print('serial port close')
ser.close()

serial port close
